In [2]:
import os
import json
filename_list = os.listdir('../data')
filename_list = sorted(filename_list, key=lambda x: (len(x), x))
tag_2 = filename_list[0]
filename_list = filename_list[1:]
print(tag_2, filename_list, len(filename_list))
rel_list = []
with open('../data/' + tag_2, 'r') as f:
    lines = f.readlines()
    lines = list(map(lambda x: x.replace('\n', ''), lines))
    f_list = list(map(lambda x: x.split(' ')[0], lines))
    rel_list = list(map(lambda x: x.split(' ')[1], lines))
    temp = sorted(zip(f_list, rel_list), key=lambda x: (len(x[0]), x[0]))
    f_list, rel_list = zip(*temp)
    f_list = list(f_list)
    rel_list = list(rel_list)
    print(f_list)
    print(rel_list)

tag_2.txt ['1_data1.txt', '2_data1.txt', '3_data1.txt', '4_data1.txt', '5_data1.txt', '6_data1.txt', '7_data1.txt', '8_data1.txt', '9_data1.txt', '10_data1.txt', '11_data1.txt', '12_data1.txt', '13_data1.txt', '14_data1.txt', '15_data1.txt', '16_data1.txt', '17_data1.txt', '18_data1.txt', '19_data1.txt', '20_data1.txt', '21_data1.txt', '22_data1.txt', '23_data1.txt', '24_data1.txt', '25_data1.txt', '26_data1.txt', '27_data1.txt', '28_data1.txt', '29_data1.txt', '30_data1.txt', '32_data1.txt', '34_data1.txt', '35_data1.txt', '36_data1.txt', '37_data1.txt', '38_data1.txt', '39_data1.txt', '40_data1.txt', '41_data1.txt', '42_data1.txt', '43_data1.txt', '44_data1.txt', '45_data1.txt', '46_data1.txt', '47_data1.txt', '48_data1.txt', '49_data1.txt', '50_data1.txt', '51_data1.txt', '52_data1.txt', '53_data1.txt', '54_data1.txt', '55_data1.txt', '56_data1.txt', '57_data1.txt', '58_data1.txt', '59_data1.txt', '60_data1.txt', '61_data1.txt', '62_data1.txt', '63_data1.txt', '64_data1.txt', '65_da

In [3]:
dataset_list = []
for filename, rel in zip(filename_list, rel_list):
    dataset = {}
    with open('../data/' + filename, 'r') as f:
        lines = f.readlines()
        filename = filename.replace('1.txt', '.txt')
        dataset['filename'] = filename
        lines = list(map(lambda x: x.replace('\n', ''), lines))
        data = list(map(lambda x: x.split(' ')[0], lines))
        label = list(map(lambda x: x.split(' ')[1], lines))
        text = ' '.join(data)
        dataset['token'] = data
        dataset['bio_label'] = label
        dataset['text'] = text
        dataset['relation'] = rel
        dataset_list.append(dataset)

In [4]:
dataset_list[0]['relation']

'110000300001100311010'

In [5]:
# 处理存在相邻B-signal、B-condition的情况
# 如：B-signal B-signal -> B-signal B-signal
for dataset in dataset_list:
    bio_label = dataset['bio_label']
    modified_bio_label = []
    pre_token = None
    for token in bio_label:
        if token == 'B-signal' and pre_token == 'B-signal':
            modified_bio_label.append('I-signal')
        elif token == 'B-condition' and pre_token == 'B-condition':
            modified_bio_label.append('I-condition')
        else:
            modified_bio_label.append(token)
        pre_token = token
    dataset['bio_label'] = modified_bio_label


In [6]:
# 把B-activity、I-activity合并为[activity]
# 把B-condition、I-condition合并为[condition]
# B-signal、I-signal保留原文中的信号词
for dataset in dataset_list:
    bio_label = dataset['bio_label']
    token = dataset['token']
    data_2_mask = []
    for word, bio in zip(token, bio_label):
        if bio == 'B-activity':
            data_2_mask.append('[activity]')
        elif bio in ['B-signal', 'I-signal']:
            data_2_mask.append(word)
        elif bio == 'B-condition':
            data_2_mask.append('[condition]')
        else:
            pass
    dataset['data_2_mask'] = data_2_mask

In [7]:
# 按照信号词在文本中的顺序提取出信号词或信号词组
# [['If'], ['If'], ['After'], ['if'], ['but', 'if'], ['After'], ['in', 'both', 'cases']]
for dataset in dataset_list:
    data_2_mask = dataset['data_2_mask']
    signal_token_list = []
    current_token_list = []
    for token in data_2_mask:
        if token not in ['[activity]', '[condition]']:
            current_token_list.append(token)
        elif current_token_list:
            signal_token_list.append(current_token_list)
            current_token_list = []
    if current_token_list:
        signal_token_list.append(current_token_list)
    dataset['signal_token_list'] = signal_token_list

In [8]:
# 将原有的的data_2_mask序列中的相邻信号词合并，并按先后顺序标号：signal-{count}
for dataset in dataset_list:
    data_2_mask = dataset['data_2_mask']
    data_2_mask_single_signal = []
    current_token_list = []
    signal_count = 0
    for token in data_2_mask:
        if token not in ['[activity]', '[condition]']:
            current_token_list.append(token)
        elif current_token_list:
            signal_count += 1 
            data_2_mask_single_signal.append(f'signal-{signal_count}')
            current_token_list = []
            data_2_mask_single_signal.append(token)  
        else:
            data_2_mask_single_signal.append(token)
    dataset['data_2_mask_single_signal'] = data_2_mask_single_signal

In [9]:
import requests
url = 'http://localhost:11434/api/generate'
for dataset in dataset_list:
    signal_token_list = dataset['signal_token_list']
    signal_token_llm_list = []
    for signal_token in signal_token_list:
        words = ' '.join(signal_token)
        data = {
            'model': 'llama3:70b-instruct',
            'prompt': f'''
            I want to extract activities, and determine the relationship between two activities from a business process text described in natural language. Below I will give you a word or phrase, denoted by [words], which connects two activities, i.e. activity1-[words]-activity2, please judge the type of relationship between two activities based on this words, including 4 types of relationship, as follows:
            [sign-selection]: indicates a selection relationship, the most common is to select an activity to be executed under certain conditions.
            [sign-successor]: indicates the direct sequential relationship between activities
            [sign-parallel]: indicates a concurrent relationship between activities.
            [sign-loop]: indicates a circular relationship between activities.
            The [words] I gave you are "[{words}]", giving the type of relationship(Just answer the type and do not output any extra content).
            ''',
            'stream': False,
        }
        try: 
            response = requests.post(url, json=data)
            result = response.json()['response']
            signal_token_llm_list.append(result)
        except:
            dataset_list_json = json.dumps(dataset_list)
            with open('datasets2.json', 'w') as f:
                f.write(dataset_list_json)
    dataset['signal_token_llm_list'] = signal_token_llm_list
    print(dataset['filename'], signal_token_llm_list)
dataset_list_json = json.dumps(dataset_list)
with open('datasets2.json', 'w') as f:
    f.write(dataset_list_json)

1_data.txt ['[sign-selection]', '[sign-selection]', '[sign-successor]', '[sign-selection]', '[sign-selection]', '[sign-successor]', '[sign-selection]']
2_data.txt ['[sign-selection]', '[sign-selection]', '[sign-selection]', '[sign-selection]', '[sign-selection]', '[sign-parallel]', '[sign-selection]']
3_data.txt ['[sign-successor]', '[sign-successor]', '[sign-selection]', '[sign-selection]', '[sign-selection]', '[sign-successor]', '[sign-selection]', '[sign-selection]', '[sign-successor]', '[sign-successor]', '[sign-successor]', '[sign-successor]', '[sign-successor]', '[sign-selection]']
4_data.txt ['[sign-parallel]', '[sign-successor]', '[sign-parallel]']
5_data.txt ['[sign-successor]', '[sign-successor]', '[sign-selection]', '[sign-selection]', '[sign-successor]', '[sign-successor]', '[sign-successor]', '[sign-selection]', '[sign-loop]', '[sign-selection]', '[sign-successor]', '[sign-successor]', '[sign-successor]', '[sign-selection]', '[sign-successor]']
6_data.txt ['[sign-successor

In [10]:
# dataset_list_json = json.dumps(dataset_list)
# with open('datasets.json', 'w') as f:
#     f.write(dataset_list_json)

In [11]:
import requests

In [12]:
url = 'http://localhost:11434/api/generate'
data = {
    'model': 'llama3:70b-instruct',
    'prompt':  '''
            I want to extract activities, and determine the relationship between two activities from a business process text described in natural language. Below I will give you a word or phrase, denoted by [words], which connects two activities, i.e. activity1-[words]-activity2, please judge the type of relationship between two activities based on this words, including 4 types of relationship, as follows:
            [sign-selection]: indicates a selection relationship, the most common is to select an activity to be executed under certain conditions.
            [sign-successor]: indicates the direct sequential relationship between activities
            [sign-parallel]: indicates a concurrent relationship between activities.
            [sign-loop]: indicates a circular relationship between activities.
            The [words] I gave you are "in both cases", giving the type of relationship(Just answer the type and do not output any extra content).
        ''',
    'stream': False,
}

response = requests.post(url, json=data)
print(response.json()['response'])

[sign-selection]
